# Chapter 13 Multivariate PMFs and Densities

## Python Code

In [34]:
from scipy.special import comb
from scipy.stats import expon
from scipy.misc import factorial
import numpy as np
import pandas as pd

### Eq 13.1

In [3]:
def multiv_dist_marbles():
    """ Probability of Y=i and B=k """
    num_yellows = 2
    num_blues = 3
    num_greens = 4
    num_total_to_pick = 4

    probas = []
    
    for i in range(num_yellows + 1):
        i_row = []
        for j in range(num_blues + 1):
            proba_i_j = comb(num_yellows, i) * \
                comb(num_blues, j) * \
                comb(num_greens, num_greens - i - j) / \
                comb(num_yellows + num_blues + num_greens, num_total_to_pick)
            i_row.append(proba_i_j)
            
        probas.append(np.array(i_row))
        
    return pd.DataFrame(np.array(probas))

In [4]:
dist_marbles = multiv_dist_marbles()
dist_marbles

,0,1,2,3
0,0.007937,0.095238,0.142857,0.031746
1,0.063492,0.285714,0.190476,0.015873
2,0.047619,0.095238,0.023810,0.000000


In [5]:
dist_marbles.values

array([[ 0.00793651,  0.0952381 ,  0.14285714,  0.03174603],
       [ 0.06349206,  0.28571429,  0.19047619,  0.01587302],
       [ 0.04761905,  0.0952381 ,  0.02380952,  0.        ]])

In [6]:
dist_marbles.values.shape

(3, 4)

In [7]:
def yellows_less_than_blues(yellows, blues):
    return yellows < blues

def proba(multiv_dist, query):
    proba_val = 0
    
    for i in range(multiv_dist.shape[0]):
        for j in range(multiv_dist.shape[1]):
            if query(i,j):
                proba_val += multiv_dist[i][j]
            
    return proba_val

proba(dist_marbles.values, yellows_less_than_blues)

0.47619047619047616

In [9]:
def simulate_marbles(
    num_pick=4,
    num_yellows=2,
    num_blues=3,
    num_greens=4,
    query=yellows_less_than_blues,
    num_experiments=10000
):
    to_choose_from = np.concatenate(
        [
            np.ones(num_yellows) * 1,
            np.ones(num_blues) * 2,
            np.ones
        ]
    )
    for i in range(num_experiments):
        np.random.choice()
    
    

## Exercises

### 1. Suppose the random pair $(X,Y)$ has the density $f_{X,Y}(s,t) = 8st$ on the triangle $\{(s,t): 0 < t < s < 1\}$.
    
(a). Find $f_X(s)$

$$
\begin{equation}
\begin{aligned}
    f_X(s) &= \int_{t=0}^{t=s}f_{X,Y}(s,t) \cdot dt && \text{integrate out $t$}\\
    &= \int_{t=0}^{t=s}8st \cdot dt \\
    &= s \int_{t=0}^{t=s} 8t \cdot dt \\
    &= s(4t^2)\big|_{t=0}^{t=s} \\
    &= 4s^3
\end{aligned}
\end{equation}
$$
    (b). Find $P(X<Y/2)$.

$$
\begin{equation}
\begin{aligned}
    P(X<Y/2) &= \int_{t=0}^{t=1}\int_{s=0}^{s=t/2}8st \cdot dsdt \\
    &= \int_{t=0}^{t=1}t4(s^2)\big|_{s=0}^{s=t/2} \cdot dt \\
    &= \int_{t=0}^{t=1}t4\big[\big(\frac{t}{2}\big)^2 - (0)^2\big] \cdot dt \\
    &= \int_{t=0}^{t=1}t^3 \cdot dt \\
    &= \frac{t^4}{4}\big|_{t=0}^{t=1} \\
    &= \frac{1}{4}
\end{aligned}
\end{equation}
$$


#### Approximation 1b

In [28]:
def approx_1b():
    """ Approximate the double integral by doing double sums. """
    sums = 0.0
    #rng = np.linspace(0,1 - 1.0/1000.0, num=1000)
    t= 0
    while t < 1.0:
        s = 0
        while s < t / 2:
            # the sum is approximated by length * width * height,
            # where the length is 0.001, width is 0.001
            # height is given as 8st
            sums += 8*s*t * 0.000001
            s += 0.001
        t += 0.001
    return sums 

In [29]:
approx_1b()

0.24916783217600103

### 2. Suppose packets on a network are of three types. In general, $40%$ of the packets are of type $A$, $40%$ have type B, and $20%$ have type $C$. We observe six packets, and denote the numbers of packets of types $A$, $B$ and $C$ by $X$, $Y$, $Z$, respectively.

#### (a) $P(X=Y=Z=2)$

$$
\begin{equation}
\begin{aligned}
    P(X=Y=Z=2) &= \frac{6!}{2!2!2!}(0.4)^2(0.4)^2(0.2)^2 && \text{multinomial dist.}\\
    &= 0.09216
\end{aligned}
\end{equation}
$$


In [32]:
factorial(6) / (factorial(2) * factorial(2) * factorial(2))* (0.4)**2 *(0.4)**2 * (0.2)**2

0.092160000000000047

#### (b) Find $Cov(X,Y+Z)$

$$
\begin{equation}
\begin{aligned}
    X + Y + Z &= 6 \\
    Y + Z &= 6 - X 
    Cov(X,Y+Z) &= Cov(X,6-X) \\
    &= E[X\cdot(6-X)]-EX \cdot E(6-X) \\
    &= E(6X-X^2)-EX\cdot (E(6) - E(X)) \\
    &= 6EX-E(X^2)-6EX+E(X)^2 \\
    &= E(X^2) - E(X)^2 \\
    &= Var(X) \\
    &= 6(0.4)(1-0.4) \\
    &= 1.44
\end{aligned}
\end{equation}
$$


#### (c) To what parametric family in this book does the distribution of $Y+Z$ belong?

Y+Z is binomially distributed, with $n=6, p=(0.4+0.2)=0.6$

### 3. Suppose $X$ and $Y$ are independent, each having an exponential distribution with means $1.0$ and $2.0$, respectively. Find $P(Y>X^2)$.

In [74]:
# approximation
(expon.rvs(scale=2.0, size=10000) \
 > (expon.rvs(scale=1,size=10000)**2)\
).sum() / 10000.0

0.65600000000000003

Note: The following integral seemed difficult to calculate by hand. So I looked at Wolfram Alpha to see if there are step-by-step solutions available for this double integral. Normally it shows solutions. However, this time, it only gave the exact solution, which makes me think that it's probably hard/impossible to calculate by hand.

$$
\begin{equation}
\begin{aligned}
    \int_{b=0}^{b=\infty}\int_{a=b^2}^{a=\infty}\frac{e^{-a/2}}{2}e^{-b}dadb &= 0.65568 && \text{Wolfram Alpha Pro}
\end{aligned}
\end{equation}
$$


In [68]:
expon.pdf(1, scale=2.0)

0.30326532985631671

In [69]:
expon.pdf(1, scale=0.5) / 2

0.1353352832366127

### 4. Suppose the pair $(X,Y)'$ has a bivariate normal distribution with mean vector (0,2) and covariance matrix $\begin{pmatrix} 1 & 2 \\ 2 & 6 \end{pmatrix}$

#### (a) Set up (but do not evaluate) the double integral for the exact value of $P(X^2 + Y^2 \leq 2.8)$

Let $\mu_1 = 0, \mu_2 = 2, \sigma_1 = 1, \sigma_2 = \sqrt{6}$, and $\rho=\begin{pmatrix} 1 & 2 \\ 2 & 6\end{pmatrix}$. 


$$
\begin{equation}
\begin{aligned}
    f_{X,Y}(s,t) &= \frac{1}{2\pi \sigma_1 \sigma_2 \sqrt{1-\rho^2}}e^{-\frac{1}{2(1-\rho^2)}\big[\frac{(s-\sigma_1)^2}{\sigma_1^2} + \frac{(t-\sigma_2)^2}{\sigma_2^2} - \frac{2\rho(s-\sigma_1)(t-\sigma_2)}{\sigma_1\sigma_2}\big]} \\
    \int_{t=-\sqrt{2.8}}^{t=\sqrt{2.8}}\int_{-\sqrt{2.8-t^2}}^{\sqrt{2.8-t^2}} f_{X,Y}(s,t) &= P(X^2 + Y^2 \leq 2.8)
\end{aligned}
\end{equation}
$$

#### (b) Using the matrix methods of Section 12.4, find the covariance matrix of the pair $U = (X+Y,X-2Y)'$. Does $U$ have a bivariate normal distribution?

$U$ does have a bivariate normal distribution because $U$ is a linear combination of bivariate random vector (X,Y)'.

$$
\begin{equation}
\begin{aligned}
    U &= \begin{pmatrix} X+Y \\ 
            X-Y \end{pmatrix} \\
    W &= \begin{pmatrix} X \\
            Y \end{pmatrix} \\
    Cov(W) &= \begin{pmatrix}
            Var(X) & Cov(X,Y) \\
            Cov(X,Y) & Var(Y) 
            \end{pmatrix} \\
    Var(X) &= 1 \\
    Cov(X,Y) &= 2 \\
    Var(Y) &= 6 \\
    A &= \begin{pmatrix} 1 & 1 \\
            1 & -1 \end{pmatrix} \\
    A' &= \begin{pmatrix} 1 & 1 \\
            1 & -1 \end{pmatrix} \\
    Cov(U) &= ACov(W)A' \\
           &= \begin{pmatrix} 11 & -5 \\
           -5 & 3 \end{pmatrix}
\end{aligned}
\end{equation}
$$


In [77]:
def compute_4b():
    a = np.array([[1, 1],[1, -1]])
    cov_w = np.array([[1,2], [2,6]])
    
    return a.dot(cov_w).dot(a.T)

compute_4b()

array([[11, -5],
       [-5,  3]])

### 5. Suppose $X$ and $Y$ are independent, and each has a $U(0,1)$ distribution. Let $V=X+Y$.

#### (a) Find $f_V$.

$$
\begin{equation}
\begin{aligned}
    f_V(t) &= \int_{s=0}^{s=t} f_X(s)f_Y(t-s)ds && \text{sum of independent R.V. is equal to convolution of the two R.V.} \\
    &= \int_{s=0}^{s=1} f_Y(t-s) ds && \text{$f_X(s)$ is $1$ if $0<s<1$, 0 otherwise} \\
\end{aligned}
\end{equation}
$$

Case 1: When $0<t<1$, $f_Y(t-s) = 1$ if $0<s<t$, $0$ otherwise. Therefore:

$$
\begin{equation}
\begin{aligned}
   \int_{0}^{t}ds &= s\Big|_0^t\\ 
   &= t
\end{aligned}
\end{equation}
$$

Case 2: When $1<t<2$, $f_Y(t-s) = 1$ if $t-1 \leq s \leq 1$, $0$ otherwise. Therefore:

$$
\begin{equation}
\begin{aligned}
   \int_{t-1}^{1}ds &= s\Big|_{t-1}^1&& \text{When $1<t<2$ } \\ 
   &= 1-(t-1) \\
   &= 2-t
\end{aligned}
\end{equation}
$$


#### (b) Verify your answer in (a) by finding $EV$ from your answer in (a) and then using the fact that $EX=EY=0.5$

$$
\begin{equation}
\begin{aligned}
    V &= X+Y \\
    EV &= E(X+Y) \\
       &= EX + EY \\
       &= 0.5 + 0.5 \\
       &= 1
\end{aligned}
\end{equation}
$$

$$
\begin{equation}
\begin{aligned}
    EV &= \int_0^t t(t)dt + \int_1^2 t(2-t)dt \\
       &= \frac{t^3}{3}\Big|_0^1 + t^2\Big|_1^2 - \frac{t^3}{3}\Big|_1^2 \\
       &= \frac{1}{3}+4-1-\Big(\frac{8}{3}-\frac{1}{3}\Big) \\
       &= 1
\end{aligned}
\end{equation}
$$


#### \* In the general population of parents who have 10-year-old kids, the parent/kid weight pairs have an exact bivariate normal distribution

#### \* Parents' weights have mean 152.6 and standard deviation 25.0.

#### \* Weights of kids have mean 62 and standard deviation 6.4.

#### \* The correlation between parents' and kids' weights is 0.4.

Use R functions (not simulation) in the following:

#### (a) Find the fraction of parents who weigh more than 160.

#### (b) Find the fraction of kidns who weigh less than 56.

#### (c) Find the fraction of parent/child pairs in which the parent weighs more than 160 and the child weighs less than 56.

#### (d) Suppose a ride aat an amusement park charges by weight, one cent for each poind of weight in the parent and child. State the exact distribution of the fee, and find the fraction of parent/child pairs who are charged less than \$2.00.

### 6. Newspapers at acertain vending machine cost 25 cents. Suppose 60% of the customers pay with quarters, 20% use two dimes and a nickel, 15% insert a dime and three nickels, and 5% deposit five nickels. When the vendor collects the money, five coins fall to the ground. Let X,Y, and Z denote the numbers of quarters, dimes, and nickels among these five coins.

#### (a) Is the joint distribution $(X,Y,Z)$ a member of a parametric family presented in this chapter? If so, which one?

#### (b) Find $P(X=2,Y=2,Z=1)$

#### (c) Find $\rho(X,Y)$.

### 7. Jack and Jill play a dice game, in which one wins \$1 per dot. There are three dice, die A, die B, and die C. Jill always rolls dice A and B. Jack always roll just die C, but he also gets credit for $90\%$ of die B. For instance, say in a particular roll A,B, and C are 3,1, and 6 respectively. Then Jill would win \$4 and Jack would get \$6.90. Let $X$ and $Y$ be Jill's and Jack's total winnings after 100 rolls. Use the Central Limit Theorem to find the approximate values of $P(X>650, Y<660)$ and $P(Y>1.06X)$.

### Hints: This will follow a similar pattern to the dice game in Section 12.7, which we win \$5 for one dot, and \$2 for two or three dots. Remember, in that example, the key was that we noticed that the pair $(X,Y)$ was a sum of random pairs. That meant that $(X,Y)$ had an approximate bivariate normal distribution, so we could find probabbilities if we had the mean vector and covariance matrix of $(X,Y)$. Thus we needed to find $EX, EY, Var(X), Var(Y)$ and $Cov(X,Y)$. We used the various properties of $E(), Var(),$ and $Cov()$.

### 8. Consider the coin game in Section 4.8. Find $F_{X_3,Y_3}(0,0)$

### 9. Suppose the random vector $X=(X_1, X_2, X_3)'$ has mean (2.0, 3.0, 8.2)' and covariance matrix $\begin{pmatrix} 1 & 0.4 & -0.2 \\ & 1 & 0.25 \\ & & 3 \end{pmatrix}$

#### (a) Fill in the missing entries.

#### (b) Find $Cov(X_1,X_3)$

#### (c) Find $\rho(X_2, X_3)$.

#### (d) Find $Var(X_3)$.

#### (e) Find the covariance matrix of (X_1 + X_2, X_2 + X_3)'.

#### (f) If in addition we know that $X_1$ has a normal distribution, find $P(1<X_1<2.5)$, in terms of $\phi()$.

#### (g) Consider the random variable $W=X_1+X_2$. Which of the following is true? _(i)_ $Var(W) = Var(X_1 + X_2)$. _(ii)_ $Var(W) > Var(X_1+X_2)$. _(iv)_  In order to determine which of the variances is the larger one, we would need to know whether the variables $X_i$ have a multivariate normal distribution. _(v)_ $Var(X_1 + X_2)$ does not exist.

### 10. Find the (approximate) output of this R code, by using the analytical techniques of this chapter:

```
count <- 0
for (i in 1:10000) {
    count1 <- 0
    count2 <- 0
    count3 <- 0
    for (j in 1:20) {
        x <- runif(1)
        if (x < 0.2) {
            count1 <- count1 + 1
        } else if (x < 0.6) count <- count2 + 1 else
                count3 <- count3 + 1
    }
    if (count1 == 9 && count2 == 2 && count3 == 9) count <- count + 1
}
cat(count/10000)
```

### 11. Use the convolution formula (13.52) to derive (7.46) for the case $r=2$. Explain your steps carefully!

### 12. The book, _Last Man Standing_, autheor D. McDonald writes the following about the practice of combining many mortgage loans into a single package sold to investors:

    Even if every single [loan] in the [package] had a 30-percent risk of default, the thinking went, the odds that most of them would default at once were arguably infinitesimal...What [this argument] missed was the auto-synchronous relationship of many loans...[If several of them] are all mortgage for houses sittingc next to each other on a beach...one strong hurricane and the [loan package] would be decimated.
    
Fill in the blank with a term from this book: The author is referring to an unwarranted assumption of _____________.

### 13. Consider the computer worm example in Section 9.3.1. Let $R$ denote the time it takes to go from state 1 to state 3. Find $f_R(v)$. (Leave your answer in integral form.)

### 14. Suppose (X,Y) has a bivariate normal distribution, with $EX = EY = 0, Var(X)=Var(Y)=1$,  and $\rho(X,Y)=0.2$. Find the following, leaving your answers in integral form:

#### (a) $E(X^2 + XY^{0.5})$

#### (b) P(Y>0.5X)

#### (c) $F_{X,Y}(0.6, 0.2)$